<a href="https://colab.research.google.com/github/CarlTeapot/Walmart-Recruiting/blob/main/model_experiment_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 29.3 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existi

In [1]:
!pip install kaggle

In [2]:
!pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.8 MB/s eta 0:00:00


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"giorgitoronjadze","key":"9c4aebca47b5575e27b4c70560a74ddf"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!pip install opendatasets
!pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.0 MB/s eta 0:00:00


In [6]:
import opendatasets as od
od.download("https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting/overview")

100%|██████████| 2.70M/2.70M [00:00<00:00, 216MB/s]


Extracting archive ./walmart-recruiting-store-sales-forecasting/walmart-recruiting-store-sales-forecasting.zip to ./walmart-recruiting-store-sales-forecasting


In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
import os
from sklearn.model_selection import TimeSeriesSplit
import pickle
from pmdarima import auto_arima

Load training data

In [8]:
train = pd.read_csv('/content/walmart-recruiting-store-sales-forecasting/train.csv.zip')  # Store, Dept, Date, Weekly_Sales, IsHoliday
features = pd.read_csv('/content/walmart-recruiting-store-sales-forecasting/features.csv.zip')  # Store, Date, Temperature, Fuel_Price, CPI, Unemployment, IsHoliday
stores = pd.read_csv('/content/walmart-recruiting-store-sales-forecasting/stores.csv')  # Store, Type, Size

print("Train columns:", train.columns.tolist())
print("Features columns:", features.columns.tolist())
print("Stores columns:", stores.columns.tolist())

Train columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']
Features columns: ['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday']
Stores columns: ['Store', 'Type', 'Size']


Data Preprocessing

In [9]:
train_merged = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
train_merged = train_merged.merge(stores, on='Store', how='left')

train_merged['Date'] = pd.to_datetime(train_merged['Date'])

train_merged = train_merged.sort_values(['Store', 'Dept', 'Date'])

train_merged.fillna({
    'MarkDown1': 0,
    'MarkDown2': 0,
    'MarkDown3': 0,
    'MarkDown4': 0,
    'MarkDown5': 0,
    'CPI': train_merged['CPI'].mean(),
    'Unemployment': train_merged['Unemployment'].mean()
}, inplace=True)

In [10]:
#unique identifier for each Store-Dept combination
train_merged['Store_Dept'] = train_merged['Store'].astype(str) + '_' + train_merged['Dept'].astype(str)

#exogenous variables
exog_columns = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'IsHoliday', 'Size']
train_exog = train_merged[exog_columns]

In [12]:
import dagshub
dagshub.init(repo_owner='CarlTeapot', repo_name='Walmart-Recruiting', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=3ed0a73b-b50b-4469-ba45-46b319f45a24&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e8b0f301e7d6040c6272f03dcbbc56058f1451d9fdaff6a7118a8e94e1ba0505




Output()

Accessing as giorgitorro

Initialized MLflow to track repo "CarlTeapot/Walmart-Recruiting"

Repository CarlTeapot/Walmart-Recruiting initialized!

In [13]:
mlflow.set_experiment('ARIMA_Training')
with mlflow.start_run(run_name='ARIMA_Preprocessing'):
    mlflow.log_param('exog_columns', exog_columns)
    mlflow.log_param('missing_value_strategy', 'fill_with_zero_or_mean')
    mlflow.log_metric('train_data_rows', len(train_merged))
    mlflow.log_metric('unique_store_dept', train_merged['Store_Dept'].nunique())

In [14]:
class ARIMAModel:
    def __init__(self, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model = None

    def fit(self, X, y):
        self.model = auto_arima(
            y,
            exogenous=X,
            order=self.order,
            seasonal_order=self.seasonal_order,
            suppress_warnings=True,
            stepwise=True
        )
        return self

    def predict(self, X):
        return self.model.predict(n_periods=len(X), exogenous=X)

 Train ARIMA Models with Cross-Validation



In [17]:
from sklearn.preprocessing import StandardScaler
import math

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('arima', ARIMAModel(order=(1, 1, 1), seasonal_order=(0, 0, 0, 0)))
])
tscv = TimeSeriesSplit(n_splits=5)

# Train and evaluate for each Store_Dept
for store_dept in train_merged['Store_Dept'].unique()[:5]:
    with mlflow.start_run(run_name=f'ARIMA_{store_dept}'):
        train_subset = train_merged[train_merged['Store_Dept'] == store_dept]
        y_train = train_subset['Weekly_Sales'].values
        X_train = train_subset[exog_columns].values

        mlflow.log_param('store_dept', store_dept)
        mlflow.log_param('arima_order', (1, 1, 1))
        mlflow.log_param('seasonal_order', (0, 0, 0, 0))

        mse_scores = []
        mae_scores = []
        for train_idx, val_idx in tscv.split(y_train):
            X_tr, X_val = X_train[train_idx], X_train[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            pipeline.fit(X=X_tr, y=y_tr)
            pred = pipeline.predict(X_val)

            mse = mean_squared_error(y_val, pred)
            mae = mean_absolute_error(y_val, pred)
            mse_scores.append(mse)
            mae_scores.append(mae)
        #average metrics
        avg_mse = np.mean(mse_scores)
        avg_rmse = math.sqrt(avg_mse)
        avg_mae = np.mean(mae_scores)
        mlflow.log_metric('avg_mse', avg_mse)
        mlflow.log_metric('avg_rmse', avg_rmse)
        mlflow.log_metric('avg_mae', avg_mae)
        print(f'Store_Dept: {store_dept}')
        print(f'Average MSE: {avg_mse}')
        print(f'Average RMSE: {avg_rmse}')
        print(f'Average MAE: {avg_mae}')

        #train on full data
        pipeline.fit(X=X_train, y=y_train)

        pipeline_path = f'model_ARIMA_{store_dept}.pkl'
        with open(pipeline_path, 'wb') as f:
            pickle.dump(pipeline, f)
        mlflow.log_artifact(pipeline_path)
        mlflow.sklearn.log_model(pipeline, f'arima_pipeline_{store_dept}')

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Store_Dept: 1_1
Average MSE: 134713625.50413644
Average RMSE: 11606.619900045682
Average MAE: 7785.545794600106


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Store_Dept: 1_2
Average MSE: 13848472.082234794
Average RMSE: 3721.3535282521593
Average MAE: 2582.726848731609


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Store_Dept: 1_4
Average MSE: 9616098.875328293
Average RMSE: 3100.9835335467833
Average MAE: 2248.3950587620498


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Store_Dept: 1_5
Average MSE: 160586643.51241204
Average RMSE: 12672.27854461904
Average MAE: 7324.899905556197


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11